In [1]:
import os

from dataset_utils import *
from geometry import mnn
from tqdm.notebook import tqdm


def detect_kpts(img_np, detector):

    kpts = detector.detect(img_np, mask=None)

    if len(kpts) == 0:
        return torch.zeros((0, 2))

    kpt_f = torch.tensor([[kp.pt[0], kp.pt[1]] for kp in kpts])
    return kpt_f


def back_project_kpts(kpts, img_rotated, rotations):
    kpts_h = torch.ones((kpts.shape[0], 3)).to(dtype=kpts.dtype)
    kpts_h[:, :2] = kpts.clone()
    Hs = torch.from_numpy(rotation_gt_Hs(img_rotated)[3 - rotations]).to(dtype=kpts.dtype)
    kpts = (Hs @ kpts_h.T)[:2].T
    return kpts


def rotate_experiment_mnn(detector, img_to_show, err_th, experiment_name):

    img_dir = "imgs/hypersim"
    files = sorted(["{}/{}".format(img_dir, fn) for fn in os.listdir(img_dir)][:img_to_show])
    means = np.zeros((3, 2))
    for file_path in tqdm(files):

        img_np_o = np.array(Image.open(file_path))
        kpts_0 = detect_kpts(img_np_o, detector)

        for rotations in range(1, 4):

            img_np_r = np.rot90(img_np_o, rotations, [0, 1])
            kpts_1 = detect_kpts(img_np_r, detector)

            kpts_1 = back_project_kpts(kpts_1, img_np_r, rotations)

            kpts_0, kpts_1, _, _ = mnn(kpts_0, kpts_1, err_th=err_th)
            mean = (kpts_1 - kpts_0).mean(dim=0)
            means[rotations - 1] += mean.numpy()

    means /= len(files)

    print()
    print(experiment_name)
    print()
    print("\t\tMean error")
    print("Rotation\tx\t\ty")
    for i, mean in enumerate(means):
        print(f"{90 + i * 90} deg\t\t{mean[0]:+.4f}\t\t{mean[1]:+.4f}")
    print()



In [2]:
import cv2 as cv
detector = cv.SIFT_create()

rotate_experiment_mnn(detector, img_to_show=10, err_th=2, experiment_name="OpenCV DoG")

  0%|          | 0/10 [00:00<?, ?it/s]


OpenCV DoG

		Mean error
Rotation	x		y
90 deg		-0.4907		+0.0004
180 deg		-0.4911		-0.4961
270 deg		+0.0016		-0.4977



In [3]:
from kornia.feature.integrated import SIFTFeature
from kornia.utils import image_to_tensor
import torch


class NumpyKorniaSiftDescriptor:

    def __init__(self, device=torch.device("cpu")):
        self.device = device
        self.sf = SIFTFeature(device=device)

    @staticmethod
    def cv_kpt_from_laffs_responses(laffs, responses):
        kpts = []
        for i, response in enumerate(responses[0]):
            yx = laffs[0, i, :, 2]
            kp = cv.KeyPoint(yx[0].item(), yx[1].item(), response.item(), angle=0)
            kpts.append(kp)
        return kpts

    def detect(self, img, mask):
        assert mask is None, "not implemented with non-trivial mask"
        if len(img.shape) == 2:
            img = img[:, :, None]
        else:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        img_t = (image_to_tensor(img, False).float() / 255.).to(device=self.device)
        laffs, responses, descs = self.sf(img_t, mask=None)
        kpts = self.cv_kpt_from_laffs_responses(laffs, responses)
        return kpts


detector = NumpyKorniaSiftDescriptor()

print("Kornia SIFT")
rotate_experiment_mnn(detector, img_to_show=10, err_th=2, experiment_name="Kornia DoG")

Kornia SIFT


  0%|          | 0/10 [00:00<?, ?it/s]


Kornia DoG

		Mean error
Rotation	x		y
90 deg		-0.4716		+0.0021
180 deg		-0.4592		-0.4659
270 deg		+0.0060		-0.4728

